<br></br>
# **tf-idf**

## **1 데이터 불러오기**
[**연간 기업결과 리포트**](https://news.samsung.com/global/samsung-electronics-announces-fourth-quarter-and-fy-2017-results)

In [ ]:
# 2017년 연간결산 리포트
f     = open('./data/News2017.txt', 'r', encoding='utf-8')
texts = f.read()
texts = texts.lower()
f.close()

In [ ]:
from nltk.tokenize import RegexpTokenizer
re_capt = RegexpTokenizer(r'[a-z]\w+')
token   = re_capt.tokenize(texts)
token[:5]

In [ ]:
from nltk.corpus import stopwords
stopwords_eng = stopwords.words('english') +\
                ['samsung','krw','quarter','full','year']
stopwords_eng[:10]

In [ ]:
texts = [txt    for txt in token   
                if txt not in stopwords_eng]

from nltk import FreqDist
import pandas as pd
token      = FreqDist(texts)
token_freq = pd.Series(token).sort_values(ascending=False)
token_freq[:10]

In [ ]:
document = ' '.join(texts)
document[:500]

<br></br>
## **2 tf idf**
연설문내 단어들의 빈도를 재조정

In [ ]:
# ! pip3 install sklearn

In [ ]:
# ! pip3 install scipy

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec   = TfidfVectorizer(stop_words='english')
transformed = tfidf_vec.fit_transform(raw_documents = [document])
transformed = np.array(transformed.todense())
transformed
# tfidf_vec.vocabulary_

In [ ]:
index_value   = {i[1]:i[0] for i in tfidf_vec.vocabulary_.items()}
fully_indexed = {index_value[column]:value  for row in transformed  
                                            for (column,value) in enumerate(row)}

token_tfidf = pd.Series(fully_indexed).sort_values(ascending=False)
token_tfidf[:10]

<br></br>
## **3 tf - idf 를 직접 구현하기**
사용자 함수를 사용하여 tf-idf 계산하기

In [ ]:
# Docs 의 list 목록을 만드는게 우선 일이다
from txtutil import tf_idf
tf_idf('it', 'it can it it', ['can', 'can','can', 'it', 'it', 'it' ])

### **1) Docs 대상문서들 목록 추출하기**

In [ ]:
stopwords_eng[:10]

In [ ]:
import re
from glob import glob
filelist    = glob('./data/News201*.txt')

docs_tokens = []
for file in filelist:
    f     = open(file, 'r', encoding='utf-8')
    texts = f.read()
    f.close()
    texts = texts.lower()
    texts = texts.replace('\n', ' ')
    tokenizer = re.compile(r'[a-z]\w+') # 영단어 추출
    tokens    = tokenizer.findall(texts)    
    tokens    = [token  for token in tokens   
                 if (len(token) > 2) and (token not in stopwords_eng)]
    docs_tokens += tokens

In [ ]:
filelist

In [ ]:
from nltk import FreqDist
import pandas as pd
pd.Series(FreqDist(docs_tokens))[:5]

### **2) Doc 대상문서에서 Token 추출하기**

In [ ]:
# 분석할 대상 데이터
f     = open('./data/News2016.txt', 'r', encoding='utf-8')
texts = f.read()
texts = texts.lower()
f.close()
texts = texts.replace('\n', ' ')
tokenizer = re.compile(r'[a-z]\w+') # 영문 데이터를 추출
tokens = tokenizer.findall(texts)

# 알파벳 2글자 이상인 단어를 대상으로 연산을 진행한다
tokens = [token  for token in tokens   
                 if (len(token) > 2) and (token not in stopwords_eng)]

token_string = " ".join(tokens)
token_string[:70]

In [ ]:
%%time
from txtutil import tf_idf
token_set = list(set(tokens))

result_dict = {}
for txt in token_set:
    result_dict[txt] = tf_idf(txt, token_string, docs_tokens)
print('Calculating is Done.')

In [ ]:
# 생성한 TF-IDF 결과를 Pandas로 출력
import pandas as pd
tfidf = pd.Series(result_dict)
tfidf.sort_values(ascending=False)[:20]